In [1]:
!unzip "IMDB_Dataset.zip"

Streaming output truncated to the last 5000 lines.
  inflating: Dataset/train/pos/5499_10.txt  
  inflating: Dataset/train/pos/55_9.txt  
  inflating: Dataset/train/pos/550_10.txt  
  inflating: Dataset/train/pos/5500_10.txt  
  inflating: Dataset/train/pos/5501_10.txt  
  inflating: Dataset/train/pos/5502_10.txt  
  inflating: Dataset/train/pos/5503_10.txt  
  inflating: Dataset/train/pos/5504_10.txt  
  inflating: Dataset/train/pos/5505_10.txt  
  inflating: Dataset/train/pos/5506_10.txt  
  inflating: Dataset/train/pos/5507_7.txt  
  inflating: Dataset/train/pos/5508_10.txt  
  inflating: Dataset/train/pos/5509_10.txt  
  inflating: Dataset/train/pos/551_8.txt  
  inflating: Dataset/train/pos/5510_10.txt  
  inflating: Dataset/train/pos/5511_8.txt  
  inflating: Dataset/train/pos/5512_7.txt  
  inflating: Dataset/train/pos/5513_7.txt  
  inflating: Dataset/train/pos/5514_9.txt  
  inflating: Dataset/train/pos/5515_7.txt  
  inflating: Dataset/train/pos/5516_7.txt  
  inflating: Data

In [90]:
import pandas as pd
import os,sys
import numpy as np
import re
import math
import matplotlib.pyplot as plt
from collections import Counter

In [91]:
#file reading. Stroring reviews in neg_train,pos_train and neg_test and pos_test
s_words = open("Dataset/stop_words.txt", "r")
neg_train=[]
pos_train=[]
neg_test=[]
pos_test=[]

path_neg="Dataset/train/neg"
all_files_neg = os.listdir(path_neg)
path_pos="Dataset/train/pos"
all_files_pos = os.listdir(path_pos)
for x in range(len(all_files_pos)):
  temp=open("Dataset/train/neg/"+all_files_neg[x],"r")
  a=temp.read()
  temp_2=open("Dataset/train/pos/"+all_files_pos[x],"r")
  b=temp_2.read()
  neg_train.append(a)
  pos_train.append(b)

path_neg_test="Dataset/test/neg"
all_files_neg_test = os.listdir(path_neg_test)
path_pos_test="Dataset/test/pos"
all_files_pos_test = os.listdir(path_pos_test)
for x in range(len(all_files_pos_test)):
  temp=open("Dataset/test/neg/"+all_files_neg_test[x],"r")
  a=temp.read()
  temp_2=open("Dataset/test/pos/"+all_files_pos_test[x],"r")
  b=temp_2.read()
  neg_test.append(a)
  pos_test.append(b)


In [92]:
#removing stop words from train data
xx=s_words.read()
def s_w_remove(data):
    temp=data.split()
    temp_2=xx.split()
    for x in range(len(temp)):
        temp[x]=temp[x].lower()
        for y in range(len(temp_2)):
            if temp[x]==temp_2[y]:
                temp[x]=""
    r_str=' '.join(temp)
    return(r_str)
for x in range(len(pos_train)):
  a=s_w_remove(pos_train[x])
  b=s_w_remove(neg_train[x])
  pos_train[x]=a
  neg_train[x]=b

In [93]:
# removing punctuation from train data
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
def pun(data):
    tempp=data.split()
    for x in range(len(tempp)):
        l=len(tempp[x])
        e_s=""
        for y in range(l):
            if tempp[x][y] not in punc:
                e_s=e_s+tempp[x][y]
        tempp[x]=e_s
    abc=' '.join(tempp)
    return abc
for x in range(len(pos_train)):
  a=pun(pos_train[x])
  b=pun(neg_train[x])
  pos_train[x]=a
  neg_train[x]=b

In [94]:
#making D and C parameters for function
# in C, 0 is negative and 1 is positive 
C=[0,1]
D=[]
for x in range(len(pos_train)):
  temp=[]
  temp.append(1)
  temp.append(pos_train[x])
  D.append(temp)
  temp_2=[]
  temp_2.append(0)
  temp_2.append(neg_train[x])
  D.append(temp_2)

In [95]:
#calculating bag of words for both positive and negative classes and storing before for better running time
neg_word={}
pos_word={}

for x in neg_train:
  arr=x.split()
  for y in arr:
    if y not in neg_word:
      neg_word[y]=1
    else:
      neg_word[y]+=1
  


for x in pos_train:
  arr=x.split()
  for y in arr:
    if y not in pos_word:
      pos_word[y]=1
    else:
      pos_word[y]+=1

def return_count(x,y):
  try:
    if x==1:
      return pos_word[y]
    if x==0:
      return neg_word[y]
  except:
    return 0


In [96]:
def TrainNaiveBayes(D,C):
  logprior={}
  loglikelihood={}
  loglikelihood[0]={}
  loglikelihood[1]={}
  for x in C:
    count=len(D)
    count_class=0

    if x==1:
      count_class=len(pos_train)
    if x==0:
      count_class=len(neg_train)

    logprior[x]=math.log(count_class/count)

    V={}

    for y in D:
      arr=y[1].split()
      for z in arr:
        if z not in V:
          V[z]=1
        else:
          V[z]+=1


    for y in V:
      temp=return_count(x,y)
      log_val=math.log((temp+1)/(count+count_class))
      loglikelihood[x][y]=log_val

    
  return logprior,loglikelihood,V

logprior,loglikelihood,V=TrainNaiveBayes(D,C)

    

In [97]:
def TestNaiveBayes(testdoc,logprior,loglikelihood,C,V):
  sum={}
  for x in C:
    sum[x]=logprior[x]
    arr=testdoc.split()
    for y in arr:
      word=y
      #print(word)
      if word in V:
        sum[x]=sum[x]+loglikelihood[x][word]
  if sum[0]>sum[1]:
    return 0
  else:
    return 1


In [98]:
#removing stop words from test data
#xx=s_words.read()
def s_w_remove(data):
    temp=data.split()
    temp_2=xx.split()
    for x in range(len(temp)):
        temp[x]=temp[x].lower()
        for y in range(len(temp_2)):
            if temp[x]==temp_2[y]:
                temp[x]=""
    r_str=' '.join(temp)
    return(r_str)
for x in range(len(pos_test)):
  a=s_w_remove(pos_test[x])
  b=s_w_remove(neg_test[x])
  pos_test[x]=a
  neg_test[x]=b

In [99]:
# removing punctuation from test data
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
def pun(data):
    tempp=data.split()
    for x in range(len(tempp)):
        l=len(tempp[x])
        e_s=""
        for y in range(l):
            if tempp[x][y] not in punc:
                e_s=e_s+tempp[x][y]
        tempp[x]=e_s
    abc=' '.join(tempp)
    return abc
for x in range(len(pos_test)):
  a=pun(pos_test[x])
  b=pun(neg_test[x])
  pos_test[x]=a
  neg_test[x]=b

In [100]:
# making D (combines dataset for test reviews. 1 is positive class 0 is negative)
D_test=[]
for x in range(len(pos_test)):
  tempp=[]
  tempp.append(1)
  tempp.append(pos_test[x])
  D_test.append(tempp)
  tempp_2=[]
  tempp_2.append(0)
  tempp_2.append(neg_test[x])
  D_test.append(tempp_2)

In [101]:
#returning and storing values from test function in return_vals
return_vals=[]
c=0
for x in range(len(D_test)):
  temp=TestNaiveBayes(D_test[x][1],logprior,loglikelihood,C,V)
  return_vals.append(temp)

In [102]:
#calculating accuracy
acc_count=0
for x in range(len(D_test)):
  if return_vals[x]==D_test[x][0]:
    acc_count+=1
print("Accuracy:",acc_count/25000*100)

Accuracy: 83.516


In [103]:
#part two
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [104]:
#making train and test variables for both test and train. x contains reviews, y contains class labels.
x_train=[]
y_train=[]
x_test=[]
y_test=[]
for x in range(len(D)):
  y_train.append(D[x][0])
  x_train.append(D[x][1])
  
for x in range(len(D_test)):
  x_test.append(D_test[x][1])
  y_test.append(D_test[x][0])

In [105]:
vectorizer = CountVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [106]:
classifier = MultinomialNB()
classifier.fit(x_train,y_train)
y_pred = classifier.predict(x_test)

In [107]:
accuracy=accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
print("Accuracy: ",accuracy)
print()
print("confusion matrix:",confusion)

Accuracy:  0.82732

confusion matrix: [[11002  1498]
 [ 2819  9681]]
